In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

In [1]:
def ColumnsExtractor_CNN(data):
    reduced_data_CNN = pd.read_csv("../data/ordered_by_ccnum_and_day.csv") #data_analysis.ipynb에서 생성하세요
    reduced_data_CNN = data[data["CC_NUM", "DAY", "AMT", "IS_FRAUD"]]
    reduced_data_CNN = reduced_data_CNN["IS_FRAUD"].astype("int")
    reduced_data_CNN.to_csv("reduced_data_CNN.csv", index=False) #디버깅용
    return reduced_data_CNN

In [6]:
def TableConverter(reduced_data, gamma):
    #reduced_data는 데이터프레임이다
    #입력DF의 행은 4개의 열을 가졌고 하나의 행은 하나의 거래이다. (CC_NUM이 아니다)
    #출력DF의 행은 하나의 CC_NUM에 대한 정보이고 열은 CC_NUM, DAY1, DAY2, ..., FRAUD_TODAY1, FRAUD_TODAY2, ... 형식이다
    #DAY1, DAY2, ... 열에는 각 DAY의 AMT가 들어간다
    df_ccnum = pd.DataFrame(reduced_data["CC_NUM"].unique(), columns=["CC_NUM"])
    df_amt = np.full((df_ccnum.shape[0], reduced_data["CC_NUM"].value_counts().max()), -1) # -1로 채워진 array (밑에 df로 변환하게 됨)
    list_daycount = []
    for i in range(reduced_data["CC_NUM"].value_counts().max()):
        list_daycount.append("DAY%d" %(i+1))
    
    df_amt = pd.DataFrame(df_amt, columns=list_daycount)
    
    #df_fraudtoday는 열의 갯수가 df_amt의 열의 갯수에서 (gamma - 1)값을 뺀 것이다
    df_fraudtoday = np.full((df_ccnum.shape[0], len(list_daycount) - (gamma - 1)), -1)
    list_fraudtodaycount = []
    for i in range(df.fraudtoday.shape[1]):
        list_daycount.append("FRAUD_TODAY%d" %(i+1))
        
    df_fraudtoday = pd.DataFrame(df_fraudtoday, columns=list_fraudtodaycount)
    
    #여기까지 오면 df 틀은 완성이다. 단 CC_NUM컬럼은 이미 값이 채워져 있는 상태
    #df_amtarray = (reduced_data["AMT"].to_frame()).T
    #CC_NUM과 DAY순으로 정렬된 AMT들이 한 행으로 쭉 정렬된 모습
    
    idx_count = 0 #CC_NUM의 인덱스를 가리킨다
    num_count = 1 #DAY1, DAY2, ...를 가리킨다
    
    for i in range(reduced_data.shape[0]):
        ccnum_unique_list = (reduced_data["CC_NUM"].unique()).tolist() #idx_count로 요소 접근
        #ccnum_unique_list는 CC_NUM의 종류가 정렬되어 나열된 리스트이다.
        
        if(ccnum_unique_list[idx_count] != reduced_data.loc[i, "CC_NUM"]):
            idx_count += 1
            num_count = 1
        
        df_amt.loc[idx_count, "DAY%d" %(num_count)] = reduced_data.loc[i, "AMT"]
        num_count += 1
        
    #여기까지 오면 df_ccnum과 df_amt가 채워졌다.
    
    ccnum_fraud = reduced_data[reduced_data["CC_NUM", "IS_FRAUD"]]
    current_ccnum = ccnum_fraud.loc[gamma - 1, "CC_NUM"]
    
    df_fraudtoday_row = 0
    df_fraudtoday_col = 0
    
    for ccnum_fraud_row in range(gamma - 1,ccnum_fraud.shape[0]):
        if(ccnum_fraud.loc[ccnum_fraud_row, "CC_NUM"]!=current_ccnum): #for문을 반복하게되면 현재 CC_NUM을 직전 CC_NUM과 비교한다
            df_fraudtoday_row += 1
            df_fraudtoday_col = 0
        df_fraudtoday.loc[df_fraudtoday_row, "FRAUD_TODAY%d" %(df_fraudtoday_col+1)] = ccnum_fraud.loc[ccnum_fraud_row, "IS_FRAUD"]
        df_fraudtoday_col += 1
        current_ccnum = ccnum_fraud.loc[ccnum_fraud_row, "CC_NUM"] # 현재 CC_NUM을 current_ccnum에 대입(for문 다시 돌아가면 ccnum_fraud_row는 1이 더해짐)
        
    data_converted = pd.concat([df_ccnum, df_amt, df_fraudtoday], axis = 0)
    data_converted.to_csv("data_converted.csv", index=False) #디버깅용
    return data_converted


In [8]:
#isLearn이 True인 경우, 입력data는 data_converted이며 출력값은 2열로 이뤄진 sequence이름의 df이다
#isLearn이 False인 경우, 입력data는 1열로 이뤄진 gamma일 동안의 amt인 df이며 출력값은 diff값인 스칼라이다.

def DifferenceGenerator(data_converted, isLearn, gamma, mu):
    differenceSequence = pd.DataFrame([], columns=["DIFF", "FRAUD_TODAY"])
    
    if(isLearn):
        reduced_data_CNN = pd.read_csv("../data/ordered_by_ccnum_and_day.csv")
        #sub_sequence_list = []
        day_col = gamma - 1 #day부분 column index 초깃값
        fraudtoday_col = 1 + reduced_data_CNN["CC_NUM"].value_counts().max() #FRAUD_TODAY부분 column index 초깃값
        
        for conv_row_idx in range(data_converted.shape[0]):
            day_col = gamma - 1 #day부분 column index 초깃값
            fraudtoday_col = 1 + reduced_data_CNN["CC_NUM"].value_counts().max() #FRAUD_TODAY부분 column index 초깃값
            for k in range((reduced_data_CNN["CC_NUM"].value_counts().max()) - (gamma - 1)): #k는 처음 위치로부터의 column의 변위
                diff_value = 0
                
                if(data_converted.loc[conv_row_idx, "DAY%d" %(day_col + k - 1)] == -1): #현재 k가 가리키는 day의 amt값이 -1이면 다음 행으로
                    break;
                
                for j in range(-gamma + 2, 1): # -gamma + 2, -gamma + 3, ..., -2, -1, 0
                    if(j != 0): #k가 가리키고 있는 amt에 mu만큼의 가중치를 곱함. 
                        prev_value = data_converted.loc[conv_row_idx, "DAY%d" %(day_col + k - 1 + j - 1)]
                        present_value = prev_value = data_converted.loc[conv_row_idx, "DAY%d" %(day_col + k + j - 1)]
                        diff_value += abs(present_value - prev_value)
                    elif (j == 0):
                        prev_value = data_converted.loc[conv_row_idx, "DAY%d" %(day_col + k - 1 + j - 1)]
                        present_value = prev_value = data_converted.loc[conv_row_idx, "DAY%d" %(day_col + k + j - 1)]
                        weighted = abs(present_value - prev_value)
                        weighted = weighted * mu
                        diff_value += weighted
                
                # 하나의 subsequence에 대한 diff의 합이 구해졌다. (diff_value)
                one_sequence = pd.DataFrame({"DIFF": diff_value, "FRAUD_TODAY":data_converted.loc[conv_row_idx, "FRAUD_TODAY%d" %(fraudtoday_col + k - 1)]})
                differenceSequence = pd.concat([differenceSequence, one_sequence], axis=0)
        
        return differenceSequence
    else: 
        #gamma -1 일 전부터 오늘까지 기록되었던 AMT들이 gamma행x1열 형태로 들어옴
        gamma_days = data_converted
        diff_value2 = 0
        for i in range(1, gamma):
            if(i != (gamma - 1)):
                prev_value2 = gamma_days.loc[i - 1]
                present_value2 = gamma_days.loc[i - 1]
                diff_value2 += abs(present_value2 - prev_value2)
            elif(i == (gamma - 1)):
                prev_value2 = gamma_days.loc[i - 1]
                present_value2 = gamma_days.loc[i - 1]
                weighted2 = abs(present_value2 - prev_value2)
                weighted2 = weighted2 * mu
                diff_value2 += weighted2
        return diff_value2 #이 때의 출력은 diff_value2라는 스칼라 값이다
        

In [3]:
def DifferenceforFraud(data, isLearn, gamma, delta2, omega, mu):
    pass
#미완성